In [1]:
%run data_r.ipynb
import numpy as np
from sklearn.model_selection import train_test_split

#Data Parameters
train_image_folder = "/datasets/ee285f-public/PascalVOC2012/JPEGImages/"
train_annot_folder = "/datasets/ee285f-public/PascalVOC2012/Annotations/"
path_to_weight     = './data/yolov2weights.h5'
#Genaral Parameters

ANCHORS            = np.array([1.07709888,  1.78171903,2.71054693,  5.12469308,5.48531347,  8.11011331])
IMAGE_H, IMAGE_W   = 416, 416
GRID_H,  GRID_W    = 13 , 13
TRUE_BOX_BUFFER    = 50
BOX                = int(len(ANCHORS)/2)
CLASS              = 20

#Loss Function Parametes
LAMBDA_NO_OBJECT   = 1.0
LAMBDA_OBJECT      = 5.0
LAMBDA_COORD       = 1.0
LAMBDA_CLASS       = 1.0

#Training Model Parameters
BATCH_SIZE         = 32
EPOCHS             = 50
LEARNING_RATE      = 0.5e-4
BETA_1             = 0.9
BETA_2             = 0.999
EPSILON            = 1e-08
DECAY              = 0.0
SAVE_WEIGHT_D      = 'save/weightst.{epoch:02d}.hdf5'

#Architecure Parameters
nb_conv            = 22
normalize          = lambda x: x/255.



Using TensorFlow backend.


In [2]:
%run model_r.ipynb
%run weight_r.ipynb
%run loss_r.ipynb
images, seen_train_labels = parse_annotation(train_annot_folder,train_image_folder, labels=LABELS)
train_image, val_image   = train_test_split(images, test_size=0.1, random_state=42)
generator_config          = {'IMAGE_H': IMAGE_H,'IMAGE_W': IMAGE_W,'GRID_H': GRID_H,'GRID_W': GRID_W,
                             'LABELS': LABELS,'ANCHORS': ANCHORS,'BATCH_SIZE': BATCH_SIZE,'TRUE_BOX_BUFFER' : TRUE_BOX_BUFFER,}
train_batch_generator     = SimpleBatchGenerator(train_image, generator_config, norm=normalize, shuffle=True)
model, true_boxes         = define_YOLOv2(IMAGE_H,IMAGE_W,GRID_H,GRID_W,TRUE_BOX_BUFFER,BOX,CLASS,trainable=False)
model                     = set_pretrained_weight(model,nb_conv, path_to_weight)
layer                     = model.layers[-4] 
initialize_weight(layer,sd=1/(GRID_H*GRID_W)) 


In [ ]:
#Set up experiment
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD, Adam, RMSprop
from keras import losses
import os
def custom_loss(y_true, y_pred):
    return(custom_loss_core(y_true,y_pred,true_boxes,GRID_W,GRID_H,BATCH_SIZE,
            ANCHORS,LAMBDA_COORD,LAMBDA_CLASS,LAMBDA_NO_OBJECT, LAMBDA_OBJECT))

dir_log = "logs/"
try:
    os.makedirs(dir_log)
except:
    pass
generator_config['BATCH_SIZE'] = BATCH_SIZE
early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, mode='min', verbose=1)
checkpoint = ModelCheckpoint(SAVE_WEIGHT_D,save_weights_only = True,monitor='loss',
                              verbose=1, save_best_only=True, mode='min', period=1)
optimizer  = Adam(lr=LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON, decay=DECAY)

losses.custom_loss = custom_loss
model.compile(loss=custom_loss, optimizer=optimizer)

history = model.fit_generator(generator = train_batch_generator, steps_per_epoch = len(train_batch_generator), epochs = EPOCHS,
                              verbose = 1,callbacks = [early_stop, checkpoint], max_queue_size   = 3)


Epoch 1/50


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:104: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


359/359 [==============================] - 253s 706ms/step - loss: 4.4383

Epoch 00001: loss improved from inf to 4.43830, saving model to save/weightst.01.hdf5
Epoch 2/50
359/359 [==============================] - 270s 751ms/step - loss: 2.8420

Epoch 00002: loss improved from 4.43830 to 2.84201, saving model to save/weightst.02.hdf5
Epoch 3/50
359/359 [==============================] - 280s 780ms/step - loss: 2.3849

Epoch 00003: loss improved from 2.84201 to 2.38485, saving model to save/weightst.03.hdf5
Epoch 4/50
359/359 [==============================] - 280s 781ms/step - loss: 2.0912

Epoch 00004: loss improved from 2.38485 to 2.09117, saving model to save/weightst.04.hdf5
Epoch 5/50
359/359 [==============================] - 275s 765ms/step - loss: 1.9083

Epoch 00005: loss improved from 2.09117 to 1.90832, saving model to save/weightst.05.hdf5
Epoch 6/50
359/359 [==============================] - 284s 792ms/step - loss: 1.7665

Epoch 00006: loss improved from 1.90832 to 1.7665

In [ ]:
#POST TRAINING ANALISYS
import numpy as np
import matplotlib.pyplot as plt
loss = history.history['loss']
x = np.arange(0, len(loss))
y = np.array(loss)
fig, ax = plt.subplots()
line, = ax.plot(x, y, label='Training Loss')
line.set_linestyle('-')
line.set_linewidth(2)
ax.set_xlabel('Epoch #')
ax.set_ylabel('Loss')


ax.legend()
plt.show()